"""
# Meeting Minutes Generator From Audio File

**Author:** [Pouria Ebrahimnezhad]  
**Date:** [14-03-2025]  
**Description:** This Jupyter Notebook uses a Hugging Face audio model to transcribe city council meeting audio recordings
and generate structured meeting minutes in Markdown format.

## Installation
Ensure dependencies are installed:
```bash
pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2
```

## Usage
1. **Specify the audio file path** in the `AUDIO_FILE` variable.
2. **Run the notebook** to process the audio and generate minutes.
3. **Review and edit the Markdown output** as needed.
4. **Save or share the meeting minutes.**

## Example Markdown Output
```markdown
# City Council Meeting - [Date]

## Attendees
- [List of Attendees]

## Agenda
1. [Agenda Item 1]
2. [Agenda Item 2]

## Key Decisions
- [Decision 1]
- [Decision 2]

## Action Items
- [Task 1]
- [Task 2]
```
"""

In [5]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

In [26]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, AutoModelForSpeechSeq2Seq, AutoProcessor
import torch
import re
from IPython.display import Markdown

In [7]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [4]:
# Connect to Colab to Google Drive and locate the Audio File

drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/llms/Seattle-council-extract.mp3"

Mounted at /content/drive


In [8]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [10]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [11]:
# Use the Whisper OpenAI model to convert the Audio to Text

audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

♪ Get the meeting started. Good afternoon, everyone. The February 8th, 2021 meeting of the Seattle City Council will now come to order. It is 2.02 p.m. I'm Lorena Gonzalez, president of the council. Will the clerk please call the roll? Council member Mosqueda? Present. Council member Peterson? Here. Council member Sawant? Here. Thank you. Council member Strauss? Present. Council member Herbold? Here. Council member Juarez? Here. Council member Lewis? Present. Council member Morales? Here. Council president Gonzalez? Present. Nine present. Thank you, madam clerk. All right, folks, we're going to move right into presentations. Council member Lewis has two presentations for members of the viewing public today. The first presentation, again, is from council member Lewis. He has a proclamation honoring Tony Lee that he will present first, and then we will open up the floor to comments. So, council member Lewis, I'm going to hand the floor over to you to walk us through the first proclamatio

In [12]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Seattle council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [13]:
# define the contization to reduce the memory impact of the models

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [15]:
# cerate a tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
# setting the padding
tokenizer.pad_token = tokenizer.eos_token
# pass in the message and the entire transcript and use GPU
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
# use streaming
streamer = TextStreamer(tokenizer)
# create the model passing the model and, GPU utilization and the quantization config from above
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)


outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Seattle council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
♪ Get the meeting started. Good afternoon, everyone. The February 8th, 2021 meeting of the Seattle City Council will now come to order. It is 2.02 p.m. I'm Lorena Gonzalez, president of the council. Will the clerk please call the roll? Council member Mosqueda? Present. Council member Peterson? Here. Council member Sawant? Here. Thank you. Council member Strauss? Present. Council member Herbold? Here. Council member Juarez? Here. Council member Lewi

In [28]:
response = tokenizer.decode(outputs[0])
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Seattle council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
♪ Get the meeting started. Good afternoon, everyone. The February 8th, 2021 meeting of the Seattle City Council will now come to order. It is 2.02 p.m. I'm Lorena Gonzalez, president of the council. Will the clerk please call the roll? Council member Mosqueda? Present. Council member Peterson? Here. Council member Sawant? Here. Thank you. Council member Strauss? Present. Council member Herbold? Here. Council member Juarez? Here. Council member Lewis? Present. Council member Morales? Here. Council president Gonzalez? Present. Nine present. Thank you, madam clerk. All right, folks, we're going to move right into presentations. Council member Lewis has two presentations for members of the viewing public today. The first presentation, again, is from council member Lewis. He has a proclamation honoring Tony Lee that he will present first, and then we will open up the floor to comments. So, council member Lewis, I'm going to hand the floor over to you to walk us through the first proclamation. Thank you so much, madam president. Today we honor the late Tony Lee. Tony resolved himself the champion to the suffering injustice for over three decades, and his staunch advocacy for the marginalized communities of the state as a former state house representative and lobbyist, when executive director at Solid Ground earned him the great respect of everybody that he worked with in the community, and at one time even the informal title of the conscience of Washington. Further than any accolades, formal or informal, Tony's work was marked, too, by official honors and awards, and these included a lifetime achievement award presented by the Seattle Human Services Coalition in 2018 for his service on the steering committee for the organization's entire 30-year existence, as well as the Goldmark Award for exceptional leadership in social justice advocacy from the Washington Legal Foundation. Tony's dedication to selfless advocacy has and will continue to earn his name and praise and respect across the state of Washington and the city of Seattle. His legacy is a reminder to us here today of why we do this work, and indeed why, as chair of the Select Committee of Homelessness Strategies and Investments, we follow in the footsteps of the path that Tony blazed in helping the people of the city of Seattle and the state of Washington live with dignity and live healthy lives. And I'm humbled today to present this proclamation and even more to take up with you all, my fellow colleagues and council members, his eternal call to action. So now I present in full a proclamation honoring the late Tony Lee for serving Seattle's most vulnerable communities. Whereas Tony Lee dedicated his life and career to advocate for marginalized communities across Washington state, and whereas he was a beloved champion for immigrant and low-income communities in the Washington state legislature, and whereas his career is marked by countless awards and accolades commending his work and leadership on social justice, and whereas his kind-hearted and collaborative nature always showed the importance of humility and appreciation of others, and whereas Tony's work spanned from establishing protections for immigrant and low-income persons to expanding education, equity, creating jobs, fighting for equal housing opportunity, and much more, and whereas Tony Lee passed away on November 12th of 2020, and whereas Tony Lee's work will live on and continue to be impactful due to his many years of serving as a strong mentor and inspiration and a model to current and future advocates, organizers, and especially people experiencing and directly impacted by poverty and racism by encouraging all people to speak truth to power and share their stories to push for change, whereas we recognize that Tony Lee's accomplishments represent an exemplary model for work that must continue, and whereas we are committed to carrying on that important work to create better futures for all communities, and whereas we make this commitment with the endurance and urgency required to make meaningful progress for the people of Seattle, and whereas we are proud to make the commitment in honor and memory of Tony Lee. Now, therefore, the mayor and the Seattle City Council proclaim Friday, February 12th, 2021 to be Tony Lee Day. I also wanna thank Parker Dawson on my staff for helping assemble this proclamation, as well as a number of colleagues who submitted content in preparation for this, and really happy to bring this forward and invite additional comments from colleagues. So thank you so much. Thank you, Council Member Lewis. Are there any other comments on the proclamation? Council Member Mosqueda, please. Thank you very much, Madam President. Thank you again, Council Member Lewis, for your office taking the helm in drafting this proclamation. It has been an honor of mine to be able to have worked side-by-side with Tony in the years that I was in the state capitol, and not only work side-by-side with him, but learn from him as a mentor. And as somebody who was working for the Children's Alliance at the time and then the State Labor Council, Tony constantly reminded us of how to do this work, to work within bellies of the beast and still remain committed to the same moral compass that drives us every single day to change policy, and that you can work within these institutions to fight for change and have labels like lobbyists and do good things, because we need folks who are bringing folks into those bellies of the beast to fight for equity and to make sure that generational poverty is never experienced. Tony did it in a way that was not only passionate, but a way that was impactful. He won so many policy changes and he made the lives better for so many people. And I'm just incredibly honored that I had the chance to work with him and will always remember his laugh. I also wanna share a note of solidarity and both strength and condolences and incredible appreciation from Euline Niao, who is working as an assemblywoman in New York's state legislature. She is wishing that she could be here with us today and also celebrate, but wanted to just say from coast to coast, we are all incredibly honored, incredibly proud to have known Tony and his work continues on as the resolution says, Tony Lee or proclamation says, Tony Lee's work will live on and continue to be impactful due to as many years of serving as a strong mentor and inspiration and model to current and future advocates, organizers, and especially people experiencing directly and directly impacted by poverty and racism by encouraging all people to speak truth to power and share their stories to push for change. That's what Tony Lee did. And we are incredibly honored to have learned from one of the greats. Thank you very much for putting this forward and incredibly happy to be here and to be with all of you, especially Angela, with you as we celebrate Tony's life this week. Thank you very much. Thank you, Council Member Mosqueda for those comments. Are there any additional comments? Council Member Herbold, please. Thank you. I was really honored to be in the presence of Tony Lee when he was recognized at the OCA Golden Circle Awards a bit ago, that's the Organization for Chinese Americans. I will always hold that last hug with Tony and hold the roaring applause for his legacy in my heart to remind me to always be as he was with the people. And Council Member Mosqueda mentioned Tony's boisterous laugh. That is another really important reminder to do the work that we do with joy and with love. And I will always think of Tony, not just about the 30 year legacy that he has created in the lives that he's changed, but his laugh and that reminder to do our work with joy and love. Thank you. Well said, Council Member Herbold. Any other comments on the proclamation?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of the Seattle City Council Meeting**
==============================================

**Summary**
----------

The Seattle City Council met on February 8th, 2021, at 2:02 p.m. to honor the late Tony Lee with a proclamation. Council Member Lewis presented the proclamation, highlighting Tony's dedication to advocating for marginalized communities and his impact on social justice. Colleagues shared their experiences and appreciation for Tony's work, emphasizing his legacy and commitment to creating better futures for all communities.

**Attendees**
------------

*   Lorena Gonzalez, Council President
*   Council Members:
    *   Mosqueda
    *   Peterson
    *   Sawant
    *   Strauss
    *   Herbold
    *   Juarez
    *   Lewis
    *   Morales

**Key Discussion Points**
-------------------------

*   Council Member Lewis presented a proclamation honoring Tony Lee for his dedication to advocating for marginalized communities and his impact on social justice.
*   Colleagues shared their experiences and appreciation for Tony's work, emphasizing his legacy and commitment to creating better futures for all communities.
*   The proclamation was commended for its recognition of Tony's work and its impact on the city of Seattle and the state of Washington.

**Takeaways**
------------

*   Tony Lee's legacy and commitment to social justice will continue to inspire and motivate the Seattle City Council.
*   The proclamation honors Tony's dedication to advocating for marginalized communities and his impact on social justice.
*   Colleagues are committed to carrying on Tony's work and creating better futures for all communities.

**Action Items**
----------------

*   The Seattle City Council will officially proclaim Friday, February 12th, 2021, as Tony Lee Day.
*   Council Member Lewis will work with colleagues to ensure the proclamation is properly recorded and disseminated.
*   Colleagues will continue to carry on Tony's work and create better futures for all communities.

**Owners**
----------

*   Council Member Lewis: Presented the proclamation and ensured its proper recording and dissemination.
*   Colleagues: Shared their experiences and appreciation for Tony's work, emphasizing his legacy and commitment to creating better futures for all communities.<|eot_id|>

## Alternative implementation

This variation uses an open-source model to transcribe the meeting Audio

In [23]:
AUDIO_MODEL = "openai/whisper-medium"
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(AUDIO_MODEL, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True)
speech_model.to('cuda')
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device='cuda',
)

Device set to use cuda


In [25]:
# Use the Whisper OpenAI model to convert the Audio to Text
result = pipe(audio_filename, return_timestamps=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [29]:
transcription = result["text"]
print(transcription)

 The meeting started. Good afternoon, everyone. The February 8th, 2021 meeting of the Seattle City Council will now come to order. It is 2.02 p.m. I'm Lorena Gonzalez, president of the council. Will the clerk please call the roll? Councilmember Mosqueda? Present. Councilmember Peterson? Here. Councilmember Sawant? Here. Councilmember... Here. Thank you. Councilmember Strauss? Present. Councilmember Herbold? Here. Councilmember Juarez? Here. Councilmember Lewis? Present. Councilmember Morales? Here. Councilpresident Gonzalez? Present. Nine present. Thank you, Madam Clerk. All right, folks, we're going to move right into presentations. Councilmember Lewis has two presentations for members of the viewing public today. The first presentation, again, is from Councilmember Lewis. He has a proclamation honoring Tony Lee that he will present first, and then we will open up the floor to comments. So Councilmember Lewis, I'm going hand the floor over to you to walk us through the first proclamat